<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

### Load and clean data

In [0]:
# Load data
import requests

r = requests.get('https://www.gutenberg.org/files/100/100-0.txt')

In [12]:
text = r.content.decode('utf-8')
text[:500]

'\ufeff\r\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever.  You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org.  If you are not located in the United States, you’ll\r\nhave to check the laws of the country where '

In [13]:
# remove some of the extra characters
text = text.replace('\r', '')
text[:500]

'\ufeff\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\nShakespeare\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever.  You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org.  If you are not located in the United States, you’ll\nhave to check the laws of the country where you are l'

In [25]:
# remove the preface so we have just Shakespear's works (keeping table of contents for now)
text = text[827:]
text[:500]

'The Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n               THE SONNETS\n\n               ALL’S WELL THAT ENDS WELL\n\n               THE TRAGEDY OF ANTONY AND CLEOPATRA\n\n               AS YOU LIKE IT\n\n               THE COMEDY OF ERRORS\n\n               THE TRAGEDY OF CORIOLANUS\n\n               CYMBELINE\n\n               THE TRAGEDY OF HAMLET, PRINCE OF DENMARK\n\n               THE FIRST PART OF KING HENRY THE FOURTH\n\n               THE SECOND PART OF KING '

In [26]:
len(text)

5572325

### Encode the data as sequences of characters

In [0]:
# Encode Data as Chars

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [29]:
# break the text into character sequences

maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is maxlen chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1114457


In [32]:
# Create X & y
import numpy as np

X = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i,t,char] = 1
        
    y[i, next_char[i]] = 1

print(X.shape)
print(y.shape)

(1114457, 40, 105)
(1114457, 105)


### Define functions for previewing predictions during the training

From the lecture notebook.

In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    # Only print a preview every 5 epochs
    if (epoch % 5 == 0):
      print()
      print('----- Generating text after Epoch: %d' % epoch)
      
      start_index = random.randint(0, len(text) - maxlen - 1)
      
      generated = ''
      
      sentence = text[start_index: start_index + maxlen]
      generated += sentence
      
      print('----- Generating with seed: "' + sentence + '"')
      sys.stdout.write(generated)
      
      for i in range(400):
          x_pred = np.zeros((1, maxlen, len(chars)))
          for t, char in enumerate(sentence):
              x_pred[0, t, char_int[char]] = 1
              
          preds = model.predict(x_pred, verbose=0)[0]
          next_index = sample(preds)
          next_char = int_char[next_index]
          
          sentence = sentence[1:] + next_char
          
          sys.stdout.write(next_char)
          sys.stdout.flush()
      print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

### Build and fit the model

In [0]:
# build the LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [50]:
# fit the model
model.fit(X, y,
          batch_size=32,
          epochs=50,
          callbacks=[print_callback])

Epoch 1/50
34826/34827 [============================>.] - ETA: 0s - loss: 2.0075
----- Generating text after Epoch: 0
----- Generating with seed: "d we create, in absence of ourself,
    "
d we create, in absence of ourself,
    These bett!
  KINE. There some, the Pisius
    Ale untidcest givenn that it swarniar,
    None.

ALMICO.
      Lay, I how discher's, thish dimenor my buss
    And surd anflesh, not lood than her denty,
Th’e wet not delisg toomancy crick a bul
    Theirath affeny wasted the Casbant,
    Lexter couss as dy coblougs the tongcted.

THUST. Yis come sucp hem blessioned raice
the dight.

PARIOP.
What hav
34827/34827 [==============================] - 196s 6ms/step - loss: 2.0074
Epoch 2/50
34827/34827 [==============================] - 182s 5ms/step - loss: 1.7040
Epoch 3/50
34827/34827 [==============================] - 184s 5ms/step - loss: 1.6050
Epoch 4/50
34827/34827 [==============================] - 183s 5ms/step - loss: 1.5509
Epoch 5/50
34827/34827 [=========

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


That’s most certain.

FIRST POMANDO.
Our one them.

DOCTOR.
My, what, yet that, you told, a parah!

            Eneen
  ANGELDOUS OS] Grace of they will scorn; for sill stip; in Vief,
    Where his excupser service, grow on state,
    Tad to liegnisRy and one farber'd to deft.
  SUFFOLK. To in home is
    King,
34827/34827 [==============================] - 195s 6ms/step - loss: 1.3882
Epoch 22/50
34827/34827 [==============================] - 180s 5ms/step - loss: 1.3836
Epoch 23/50
34827/34827 [==============================] - 180s 5ms/step - loss: 1.3800
Epoch 24/50
34827/34827 [==============================] - 181s 5ms/step - loss: 1.3767
Epoch 25/50
34827/34827 [==============================] - 180s 5ms/step - loss: 1.3738
Epoch 26/50
34820/34827 [============================>.] - ETA: 0s - loss: 1.3711
----- Generating text after Epoch: 25
----- Generating with seed: " shore to shore, and left me breath
Noth"
 shore to shore, and left me breath
Noth make my sake, this soad the

### Generate a single prediction at a time

In [0]:
# separating some code from the on_epoch_end function earlier
def predict(prediction_model, length=400):
  """ 
  returns a random prediction from the model
  
  param length: the number of characters to generate
  """

  start_index = random.randint(0, len(text) - maxlen - 1)
  seed = text[start_index: start_index + maxlen]
  generated = seed
  
  print('----- Generating with seed: "' + seed + '"\n')
  
  for i in range(length):
    # encode the seed for the model
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(seed):
      x_pred[0, t, char_int[char]] = 1
    
    # make a prediction
    preds = prediction_model.predict(x_pred, verbose=0)[0]

    # convert back from index to character
    next_index = sample(preds)
    next_char = int_char[next_index]
    
    # shift seed for the next prediction
    seed = seed[1:] + next_char

    # save the generated character to the output
    generated += next_char

  return generated

In [0]:
start_index = random.randint(0, len(text) - maxlen - 1)
      
generated = ''

sentence = text[start_index: start_index + maxlen]
generated += sentence

print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(400):
  x_pred = np.zeros((1, maxlen, len(chars)))
  for t, char in enumerate(sentence):
    x_pred[0, t, char_int[char]] = 1
      
  preds = model.predict(x_pred, verbose=0)[0]
  next_index = sample(preds)
  next_char = int_char[next_index]
  
  sentence = sentence[1:] + next_char
  
  sys.stdout.write(next_char)
  sys.stdout.flush()
print()

In [65]:
print(predict(model, length=500))

----- Generating with seed: "oud.

DIOMEDES.
Or covetous of praise.

"



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


oud.

DIOMEDES.
Or covetous of praise.

LEONTES.
Yes, death, late.

CLOWN.
Is this capbray.

GOW, Godsan belieun

Abate stareter’ told of uniters of me that he is
      the world to had this no one. Let station is nothing.

           Enter PAGE. Not forgen your part you gone?
  OLIVER. Look I buke were penarition
    To deny lustiess and will of schoducude
Do us that answer and that must your form wound late,
Make so much hands pains to peacent vown appare,
In shills and to ridest nor thought.
You do come to yet. Making Claudio thy
f


### Save and download model so I can use it again without re-training

https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [66]:
!pip install h5py

In [0]:
# Save model to a file
model.save("shakespear_character_model.h5")

# Download from colab to my local machine, for later
from google.colab import files
files.download('shakespear_character_model.h5')

### How to load the model

In [0]:
from tensorflow.keras.models import load_model

# load model
loaded_model = load_model('shakespear_character_model.h5')

In [70]:
# demonstrate that the loaded model can make the same predictions
predict(loaded_model, length=500)

----- Generating with seed: ", and it no more merits
The tread of a m"



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


", and it no more merits\nThe tread of a men in great rest cast out,\n    As joyled them young his tomons to us,\n  RAILD anoolonalios, true.\n\n Enter Beatry and Duke the neglorien:—to me, look you ediour vain to\nSo might brother.\n\n [_Exit._]\n\n [_Exeupings._]\n\nPAROLLES.\nA villain is approal.\nI becom his fleet is peace up.       Olle here have even to a spack known ungues charge,\nA toil ear alonas. What arse sir, my kinent palace:\n More was bound his bound, crysun'd of penest.\n    Hearth of Romen to forsack'd and law not in\n    Is this trus"

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN

## Stretch Goal: tokenize each word and train a model with words instead of characters.

### Spacy tokenization

In [0]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

def get_tokens(doc_array):
    tokens = []
    for doc in tokenizer.pipe(doc_array, batch_size=500):
        tokens.append([token.text for token in doc])
    return tokens

In [0]:
# break the text up into chunks of words

chunk_length = 20  # words in each chunk

# get an array of all words
temp = get_tokens(text)

w = 0
words = []
while word < len(temp):
  for i in range(chunk_length):
    words += temp[w]
    w += 1

print(f"Text separated into {len(words)} chunks of {chunk_length} words each")

In [0]:
words[0]

### Using gensim's Dictionary structure to encode those words

In [0]:
from gensim import corpora

### Create and fit a model

### Making predictions with this model

### Saving and loading for later